In [2]:
import numpy as np
import pandas as pd

In [3]:
def clean_data(df, name_index='Date', outliers = False, name_column2clean = 'Adj Close'):
    """Cleans input dataframe und returns a cleaned dataframe, by
        filling NaNs with the previous value; if outliers = True also outliers are replaced"""
    if name_index == 'Date':
        start = pd.to_datetime(df[name_index].iloc[0], infer_datetime_format = True)
        end = pd.to_datetime(df[name_index].iloc[-1], infer_datetime_format = True)
        dates = pd.date_range(start=start, end=end)
        dates = pd.DataFrame(dates, dates, columns=['Dummy'])
    else:
        if df[name_index].dtype == type('someString'):
            print("Type error: index cannot be a string! It must be a number or a date!")
        else:
            start = df[name_index].min()
            end = df[name_index].max()
            full_range = [x for x in range(start, end+1)]
            dates = pd.DataFrame(full_range, full_range, columns=['Dummy'])
    df.set_index(name_index, inplace = True)
    
    ### Replace missing data by previous dates
    df = pd.merge(dates, df, left_index=True, right_index=True, how='left')
    df.fillna(method='ffill', inplace=True)
    df.drop(columns=['Dummy'], inplace=True)
    if outliers == True:
        replace_outliers(df, name_index=name_index, name_column2clean = name_column2clean)
    df.reset_index(inplace = True)
    ############## Bring dataframe to same structure as it had at the time of input
    df[name_index] = df['index']
    df.drop(columns=['index'], inplace = True)
    return df

In [4]:
def replace_outliers(df, name_index, name_column2clean):
    """Returns a dataframe by replacing outliers by preceding values"""
    col = name_column2clean
    original = np.array(df[col])
    for n in range(len(original)):
        original = np.array(df[col])
        shift = np.roll(original, 1)
        shift_back = np.roll(original, -1)
        diff = abs(original - shift)
        diff_back = abs(original - shift_back)

        outlier_detector = np.quantile(original, q=0.75) - np.quantile(original, q=0.25)
        if n == 0:
            ### replace outlier on first data point by second one
            if diff_back[n] > outlier_detector:
                df[col].iloc[0] = df[col].iloc[1]
        else:        
            ### replace outlier by previous data point
            if diff[n] > outlier_detector:
                df[col].iloc[n] = df[col].iloc[n-1]
    return df

In [5]:
def simulate_investment(start_date, end_date, df, name_price_column = "Adj Close", \
                        interval='monthly', amount=100, costs=0.01, TER=0.01):
    """Computes the total return of some asset (given by the df with Date, Price) starting at start_date
    ending at end_date and an invested amount of money each interval (daily, monthly, quarterly) at a cost 
    (Ordergebühr) of costs and a TER (total expense ratio yearly)
    """

    startdate = pd.to_datetime(start_date, infer_datetime_format = True)
    enddate = pd.to_datetime(end_date, infer_datetime_format = True)
    df_temp = df.copy()
    
    if (df_temp.isna().sum().sum())>0:
        print("Cleaning dataframe: ", df_temp.isna().sum().sum(), " NaNs in total.")
        df_temp = clean_data(df_temp, name_column2clean = name_price_column)
        df_temp.set_index('Date', inplace = True)
    else:
        df_temp.set_index('Date', inplace = True)
    
    ### financial things
    portfolio = {'Index':[], 'Amount':[], 'Price':[], 'Value':[]}
    value = 0.
    
    
    if interval == 'monthly':
        offset = 1
    elif interval == 'quarterly':
        offset = 3
    newoffset = offset
    date = startdate
    while (date <= enddate):
        ### price at this date
        price = df_temp['Adj Close'].loc[date]
        if interval == 'monthly':
            TER_daily = TER/12.
        else:
            print("TER not correctly implemented for other than monthly allocation!")
        if date == startdate:
            value += amount - amount*costs
        else:
            value -= TER_daily * value 
            value = price/oldprice * value 
            value += amount - amount*costs
        #portfolio.append({'Index':date, 'Amount':amount, 'Price':price, 'Value':value})
        portfolio['Index'].append(date)
        portfolio['Amount'].append(amount)
        portfolio['Price'].append(price)
        portfolio['Value'].append(value)

        #print("Investing ", amount, " € at a price of ", df_temp['Adj Close'].loc[date], " at ", date, "with actual value =", value)
        ### go to next date
        oldprice = price
        date = startdate + pd.DateOffset(months=newoffset)
        newoffset += offset
    investment = pd.DataFrame.from_dict(portfolio)
    investment.set_index('Index', inplace=True)
    return(investment)
    
    ## number of stocks to buy at given price
    #stocks = int(amount/df['Adj Close'])
    ## total money invested at given price
    #invested = stocks * df['Adj Close']
    

In [6]:
def invest_return_yearly(invest, name_amountSpent_column = 'Amount', name_valueInvestment_column = 'Value',
                        name_index = 'Index'):
    """invest has to be a pd.DataFrame with columns Amount and Value"""
    expenses = invest[name_amountSpent_column].sum()
    value = invest[name_valueInvestment_column].iloc[-1]
    years = int(str(invest.reset_index()[name_index].iloc[-1] - \
                    invest.reset_index()[name_index].iloc[0]).split(" days")[0])/365.
    annual_return = (value / expenses-1)/years*100
    return(annual_return)

In [ ]:
def prepare_portfolio_data(mindate, maxdate, portfolio):
    """Prepares and cleans data for all ETFs in portfolio and returns an array with the data"""
    range_days = int((maxdate - mindate).days) + 1
    assets_in_daterange = np.zeros((range_days, len(portfolio)))
    for n, asset in enumerate(portfolio):
        df = pd.read_csv("data/{0}.csv".format(asset))
        df = clean_data_with_outliers(df)
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
        ### fill in missing data points
        df_daterange = np.array(df.loc[(df['Date'] >= mindate) & (df['Date'] <= maxdate), 'Adj Close'])
        assets_in_daterange[:,n] = df_daterange
    ### each row represents data of one asset
    assets_in_daterange = assets_in_daterange.transpose()
    return assets_in_daterange

In [7]:
def portfolio_return_risk(assets, parameters, total_amount, name_date_column = 'Date', name_price_column = 'Adj Close'):
    """Computes the total return for a portfolio consisting of a list of assets: 
    assets: np.array of shape (num_assets, num_datapoints)
    parameters = array{[percentages], [costs], [TER]}"""
    num_assets = len(assets)
    time_interval = len(assets[0])
    returns = []
    percentage = parameters.transpose()[0]
    ### Compute yearly return for all assets
    for n in range(num_assets):
        ### The exact date the computation is done is not important, since only prices are used
        time = pd.date_range(start='1970-01-01', periods=len(assets[n]))
        df_price = pd.DataFrame({name_date_column:time, name_price_column: assets[n]})
        if n == 0:
            assets_oneYear = np.array(df_price[name_price_column])
        else:
            assets_oneYear = np.vstack((assets_oneYear , np.array(df_price[name_price_column])))
        
        amount = parameters[n][0]*total_amount
        costs = parameters[n][1]
        TER = parameters[n][2]
        r = simulate_investment(time.min(), time.max(), df_price, amount=amount, costs=costs, TER=TER);
        returns.append(invest_return_yearly(r))
    all_returns = np.array(returns)

    
    total_return = np.dot(percentage,all_returns)
    total_var = 0.
    ### Compute yearly risk for all assets
    for n in range(num_assets):
        total_var += percentage[n]**2*np.var(assets_oneYear[n])
        for m in range(n):
            total_var += 2.*percentage[n]*percentage[m]*np.cov(assets_oneYear, bias=1)[n][m]
    if num_assets == 1:
        print("Only one stock is part of portfoilio")
        return_risk = (returns[0], np.var(assets_oneYear[0]));
    elif percentage.max()==1:
        idx = np.where(percentage==1)[0][0]
        print(idx)
        print("Only one stock is part of portfolio, because of stock mix!")
        return_risk = (returns[idx], np.var(assets_oneYear[idx]))
    else:
        return_risk = (total_return, total_var);
    return(return_risk)
        

In [8]:
####
#Portfolio funktion: erhält liste von aktien und berechnet für jede aktie mean, var, cov 
#    andere Funktion: berechnet für gewisse Auswahl an Aktien das optimale Mischungsverhältnis

In [9]:
def get_lows_highs(df, top_x=5, name_price_column="Adj Close"):
    """Computes the Top X dates of lowest Adj Close price for given series"""
    

    df.sort_values(name_price_column, ascending=True, inplace = True)
    low_list = df[name_price_column].iloc[:top_x]
    high_list = df[name_price_column].iloc[-top_x:]
    
    return (low_list, high_list)

In [10]:
def pearson(assets):
    """Computes pearson correlation matrix for a list of assets in the same date interval
        assets: np.array of shape (num_assets, num_datapoints)"""
    num_assets = len(assets)
    if num_assets == 1:
        pearson = 1
    else:
        ### bias=1 means normalizing with 1/N
        cov = np.cov(assets, bias=1)
        pearson = np.zeros((num_assets, num_assets))
        for n in range(num_assets):
            for m in range(num_assets):
                pearson[n][m] = cov[n][m]/(np.sqrt(np.var(assets[n])*np.var(assets[m])))
                #pearson[n][m] = cov[n][n]/(np.var(assets[n]))
                #pearson[n][n+1] = cov[n][n+1]/(np.sqrt(np.var(assets[n])*np.var(assets[n+1])))
                #pearson[n+1][n] = cov[n+1][n]/(np.sqrt(np.var(assets[n+1])*np.var(assets[n])))
    return(pearson)

In [ ]:
def normalize(assets):
    """Normalizes all stock data
    assets: np.array of shape (num_assets, num_datapoints)"""
    num_assets = len(assets)
    for n in range(num_assets):
        Min = assets[n].min()
        Max = assets[n].max()
        assets[n] -= Min
        assets[n] /= (Max-Min)